In [1]:
import numpy as np
from numpy.linalg import svd, lstsq
from scipy.integrate import solve_ivp

import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 18
np.random.seed(123)

In [3]:
# Simulate the Lorenz System
tstart, tstop = 0, 50
dt = 0.01
timepoints = np.arange(tstart, tstop + dt, dt)

σ, ρ, β = 10, 28, 8/3
x_0 = (-8, 8, 27)

def lorenz(t, x_t, σ=σ, ρ=ρ, β=β):
    """Lorenz differential equation, returns LHS (derivatives) at a point `x_t`."""
    x, y, z = x_t
    return σ*(y - x), x*(ρ - z) - y, x*y - β*z

x = solve_ivp(lorenz, (tstart, tstop), x_0, t_eval=timepoints, rtol=1e-12, atol=1e-12).y.T
npoints, ncoords = x.shape

In [4]:
# HAVOK model of Lorenz System

# Number of shift-stacked rows
stackmax = 10

# Rank of HAVOK model
r = 10

# Eigen-time delay coordinates
H = np.empty((stackmax, npoints-stackmax))
for k in range(stackmax):
    H[k, :] = x[k:-(stackmax-k), 0] # weird - why just the x (ie 0-th) component?
    
U, S, VT = svd(H, full_matrices=False)
V = VT.T

# Derivatives (4th Order Central Difference)
dV = 1/(12*dt) * (-V[4:] + 8*V[3:-1] - 8*V[1:-3] + V[:-4])

# Trim first and last two of V that are lost in the derivative
V = V[2:-2]

# Build HAVOK Regression Model on Time Delay Coordinates
Ξ, *_ = lstsq(V, dV, rcond=None)
A = Ξ[:r-1, :r-1].T
B = Ξ[-1,   :r-1]

In [5]:
with np.printoptions(floatmode='fixed', precision=4, suppress=True):
    print(B)

[ -0.0000  -0.0000   0.0001   0.0000   0.0074  -0.0000   0.8960  -0.0005
 -44.8691]
